In [1]:
import pandas as pd 
# importer le dataset
df = pd.read_csv(r'raw_sales.csv')

In [2]:
# afficher le dataset
df.head()

,datesold,postcode,price,propertyType,bedrooms
0,2007-02-07 00:00:00,2607,525000,house,4
1,2007-02-27 00:00:00,2906,290000,house,3
2,2007-03-07 00:00:00,2905,328000,house,3
3,2007-03-09 00:00:00,2905,380000,house,4
4,2007-03-21 00:00:00,2906,310000,house,3


In [3]:
# conserver les colonnes qu'on va travailler avec
colonnes_a_conserver=['datesold','price','postcode']
nouveau_df = df.loc[:, colonnes_a_conserver]

In [4]:
# afficher le nouveau dataset
nouveau_df.head()

,datesold,price,postcode
0,2007-02-07 00:00:00,525000,2607
1,2007-02-27 00:00:00,290000,2906
2,2007-03-07 00:00:00,328000,2905
3,2007-03-09 00:00:00,380000,2905
4,2007-03-21 00:00:00,310000,2906


In [5]:
#stocker les données dans Redis dans un hash sous le nom "sales"
import redis
import json

# Se connecter à Redis
r = redis.Redis(host='localhost', port=6379,db=1)

#parcourir le nouveau_df
for index, row in nouveau_df.iterrows():
    
    # Accéder aux valeurs des colonnes pour chaque ligne
    house_id="price_house_"+str(row['postcode'])
    sale = {house_id: row['price']}   
    
    # Vérifier si la clé existe déjà
    if r.hexists('sales', row['datesold']):
        ventes = r.hget('sales', row['datesold'])
        ventes = json.loads(ventes.decode('utf-8'))
        ventes.append(sale)
    else:
        ventes = [sale]
# Stocker les données dans Redis sous forme de hash avec le nom "sales",
#les clés comme des dates(jour) et leurs valeus comme des objets json (ventes)
    r.hset('sales', row['datesold'], json.dumps(ventes))  

In [6]:
#calculer les revenus par jour et stocker les données dans Redis dans un hash sous le nom "daily_income"

# Fonction pour calculer le revenu quotidien
def calculer_revenu_quotidien():
    revenus_quotidiens = {}

    # Récupérer toutes les clés  du hash "sales"
    dates = r.hkeys("sales")

    # Parcourir les dates
    for date in dates:
        # Récupérer la valeur (ventes JSON) associée à chaque clé (date)
        ventes_json = r.hget("sales", date)

        # Convertir la valeur JSON en liste de ventes
        ventes = json.loads(ventes_json)

        revenu_quotidien = 0

        # Parcourir les ventes et accumuler le revenu quotidien
        for vente in ventes:
            for _, revenu in vente.items():
                revenu_quotidien += revenu

        # Stocker le revenu quotidien dans le dictionnaire avec la date comme clé
        revenus_quotidiens[date.decode("utf-8")] = revenu_quotidien

    return revenus_quotidiens

# appel à la focntion calculer_revenu_quotidien()
revenus = calculer_revenu_quotidien()


# Stocker les revenus quotidiens dans Redis
for date, revenu in revenus.items():
    r.hset('daily_income', date, revenu) 

In [7]:
#regrouper les revenus par mois et stocker les données dans Redis dans un hash sous le nom "monthly_income"

# Récupérer les données de revenus quotidiens
donnees_quotidiennes = r.hgetall('daily_income')

# Créer un dictionnaire pour stocker les revenus mensuels
revenus_mensuels = {}

# Parcourir les données quotidiennes et regrouper par mois
for date, revenu in donnees_quotidiennes.items():
    date = date.decode('utf-8')
    revenu = float(revenu)
    mois = date[:7]
    
    if mois in revenus_mensuels:
        revenus_mensuels[mois] += revenu
    else:
        revenus_mensuels[mois] = revenu

# Stocker les revenus mensuels dans Redis
for mois, revenu in revenus_mensuels.items():
    r.hset('monthly_income', mois, revenu)


In [8]:
#regrouper les revenus par année et stocker les données dans Redis dans un hash sous le nom "annual_income"

# Récupérer les données de revenus mensuels
donnees_mensuels = r.hgetall('monthly_income')

# Créer un dictionnaire pour stocker les revenus annuels
revenus_annuel = {}

# Parcourir les données mensuels et regrouper par l'année
for date, revenu in donnees_mensuels.items():
    date = date.decode('utf-8')
    revenu = float(revenu)
    annee = date[:4]
    
    if annee in revenus_annuel:
        revenus_annuel[annee] += revenu
    else:
        revenus_annuel[annee] = revenu

# Stocker les revenus annuels dans Redis
for annee, revenu in revenus_annuel.items():
    r.hset('annual_income', annee, revenu)